Je kunt de vraag veranderen

Kies dan voor "Clear All Outputs" en "Run All" om het hele RAG pattern te doorlopen
met een nieuwe vraag.

In [11]:
myDataQuestion = "Wie is Remco?"

In [12]:
#inititalize the notebook
import os
import textwrap
import pymongo
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

mongo_user = os.getenv("COSMOS_MONGO_USER")
mongo_pwd = os.getenv("COSMOS_MONGO_PWD")
mongo_server = os.getenv("COSMOS_MONGO_SERVER")
mongo_conn = f"mongodb+srv://{mongo_user}:{mongo_pwd}@{mongo_server}?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"


embeddings_deployment_name = "text-embedding-ada-002"
chat_deployment_name = "gpt-4"

databaseName = "hszuyd"
collectionName = "webpagecollection"

mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client[databaseName]
collection = db[collectionName]

print("*** init done! ***")


*** init done! ***


RETRIEVE

In [13]:
def vector_search(vector, num_results=3):
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": vector,
                    "path": "contentVector",
                    "k": num_results 
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results


#create an embedding for the question we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#zoek relevante pagina's op basis van de vraag
results = vector_search(questionEmbedding, 3)

retrievedData = []
print("De top 3 resultaten zijn: ")
for result in results:
    if result["similarityScore"] > 0.8:
        #retrievedData.append({"role": "system", "content": "## data \n " + result["document"]["content"]})
        retrievedData.append({"document": result["document"]["filename"],"content": result["document"]["content"]})
    print(result["document"]["filename"] + " - " + str(result["similarityScore"]))


De top 3 resultaten zijn: 
hbo-ict-999.md - 0.8602040749727652
hbo-ict-1.md - 0.7790297332317446
hbo-ict-0.md - 0.7790163027654221


AUGMENT

In [14]:
messages = [
    {
        "role": "user",
        "content": myDataQuestion
    },
    {
        "role": "system",
        "content" : f"""
            Je bent een behulpzame assistent
            Je geeft alleen antwoord op basis van de data die je hebt gekregen
            Indien je het antwoord niet kunt vinden, 
            zeg je dat je het antwoord niet weet

            ## data
            {retrievedData}
        """
    }
]

#messages.extend(retrievedData)

messages




[{'role': 'user', 'content': 'Wie is Remco?'},
 {'role': 'system',
  'content': '\n            Je bent een behulpzame assistent\n            Je geeft alleen antwoord op basis van de data die je hebt gekregen\n            Indien je het antwoord niet kunt vinden, \n            zeg je dat je het antwoord niet weet\n\n            ## data\n            [{\'document\': \'hbo-ict-999.md\', \'content\': \'Wie is Remco Brosky?\\n\\nRemco Brosky is Senior Cloud Solution Architect bij Microsoft. Hij is onderdeel van de Customer Succes Unit in Nederland in het domein "Digital and Application Innovation". Hij werkt dagelijks met klanten op het gebied van Cloud Native Application Development, Developer Experience en het Development Process.\\nDenk hierbij aan Best practices rondom het gebruik van Microsoft Azure, Generative AI, Github, Azure DevOps en diverse development diensten zoals Kubernetes en App Services. \\nHij heeft een achtergrond in software engineering en architecture en is een Microsoft

GENERATE

In [15]:
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=messages
    )

lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)





Remco Brosky is Senior Cloud Solution Architect bij Microsoft. Hij is onderdeel
van de Customer Succes Unit in Nederland in het domein "Digital and Application
Innovation". Hij werkt dagelijks met klanten op het gebied van Cloud Native
Application Development, Developer Experience en het Development Process. Hij
heeft een achtergrond in software engineering en architecture en is een
Microsoft Certified Architect en Trainer.
